In [12]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests

In [13]:
def connect_to_gs(_service_account_key):
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    credentials = service_account.Credentials.from_service_account_info(_service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection

_gc = connect_to_gs(st.secrets["gcp_service_account"])

In [14]:
def update(_gc):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()

    events = pd.DataFrame(general_response['events'])[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]
    if len(events[events['is_current'] == True]) == 0:
        print('season hasn\'t started pull placeholder data')
        gw = 0
        return False, gw
    
    gw = int(events[events["is_current"] == True]["id"])
    finished = bool(events[events["is_current"] == True]["finished"].values)
    data_checked = bool(events[events["is_current"] == True]["data_checked"].values)
    
    if finished and data_checked:
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        max_stored_gw = fetch_max_gw(_gc, gameweek_results_table, prod_google_sheet_key)
        if max_stored_gw < gw:      
            print('max stored_gw < current week')
            print('we will update here')
            return True, gw
        
        else: 
            print('data is already latest - pull from gs')
            return False, max_stored_gw
    else:
        print("its either a new week or the old week hasnt completetly finished - pull from gs")
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        return False, gw, finished, data_checked
    
update_flag, current_gw, finished, checked = update(_gc)

its either a new week or the old week hasnt completetly finished - pull from gs
finished: False
checked: False
game week: 1


In [15]:
update_flag = True

In [16]:
def fetch_manager_data(_gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # to handle numeric columns that are imported as strings
        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

def append_missing_rows(df1, df2, current_gw):
    # Extract relevant columns
    t1 = df1[["entry", "player_name"]]
    t2 = df2[["entry", "player_name"]]
    
    # Find missing rows
    missing_rows = t2[~t2['entry'].isin(t1['entry'])]
    
    if missing_rows.empty:
        return None
    
    # Add new columns
    missing_rows["event_joined"] = current_gw
    missing_rows["uno_reverse"] = "Yes"
    
    return missing_rows

def write_google_sheets_data(_gc, df, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        df_values = df.values.tolist()
        gs.values_append(sheet_name, {"valueInputOption": "RAW"}, {"values": df_values})

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

def managers_update(_gc, league_endpoint, current_gw):
    # Fetch league data using a session
    with requests.Session() as session:
        league_response = session.get(league_endpoint).json()

    # Create DataFrame from league standings
    league_data = league_response["standings"]["results"]
    df2 = pd.DataFrame(league_data)

    #convert columns to numerics
    numerical_columns = ['entry', 'event_joined']
    df1 = fetch_manager_data(_gc, managers_table, prod_google_sheet_key, numerical_columns)

    # Append missing rows and handle the result
    missing_rows = append_missing_rows(df1, df2, current_gw)
    if missing_rows is None:
        return df1[["entry", "player_name"]]

    # Write missing rows to the Google Sheet
    write_google_sheets_data(_gc, missing_rows, managers_table, prod_google_sheet_key)
    return missing_rows[["entry", "player_name"]]

manager_details = managers_update(_gc, league_endpoint, current_gw)

In [17]:
manager_details

,entry,player_name
0,1805743,Peter Wertz
1,3854998,Cole Floyd
2,4984487,Tijil Jugadoe
3,574473,Liam Smorfitt
4,2599092,Niklas Wietzorrek
5,2429570,Hethe Brinkman
6,2158039,Rhys Floyd
7,247866,Connor McDonald
8,364408,Alex Wietzorrek
9,7545,Devlin Sharp


In [18]:
def get_player_stats(player_id, live_gw_data):
    for element in live_gw_data["elements"]:
        if element["id"] == player_id:
            return element["stats"]
    return None

def analyze_picks(picks, live_gw_data):
    red_card = 0
    own_goal = 0
    penalties_missed = 0

    for player_id in picks:
        player_stats = get_player_stats(player_id, live_gw_data)
        if player_stats:
            red_card += player_stats["red_cards"]
            own_goal += player_stats["own_goals"]
            penalties_missed += player_stats["penalties_missed"]

    return red_card > 0, own_goal > 0, penalties_missed > 0


def gameweek_results_update(_gc, current_gw, manager_details):
    new_gameweek_results = pd.DataFrame()
    picks_data = pd.DataFrame()

    for entry, player_name in zip(manager_details['entry'], manager_details['player_name']):
        live_teams_endpoint = f'https://fantasy.premierleague.com/api/entry/{entry}/event/{current_gw}/picks/'
        with requests.Session() as session:
            live_teams_response = session.get(live_teams_endpoint).json()

        live_teams = live_teams_response['entry_history']

        # Create new player row
        new_player_row = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'points': live_teams['points'],
            'total_points': live_teams['total_points'],
            'event_transfers': live_teams['event_transfers'],
            'event_transfers_cost': live_teams['event_transfers_cost'],
            'points_on_bench': live_teams['points_on_bench']}
        
        # Append to new_gameweek_results using concat
        new_gameweek_results = pd.concat([new_gameweek_results, pd.DataFrame([new_player_row])], ignore_index=True)

        # Get player picks
        elements_out = [sub["element_out"] for sub in live_teams_response["automatic_subs"]]
        elements_in = [sub["element_in"] for sub in live_teams_response["automatic_subs"]]
        picks = live_teams_response["picks"]

        element_ids_modified = [pick["element"] for pick in picks if pick["element"] not in elements_out]
        element_ids_modified.extend(elements_in)

        live_gameweek_endpoint = f'https://fantasy.premierleague.com/api/event/{current_gw}/live/'
        with requests.Session() as session:
            live_gameweek_response = session.get(live_gameweek_endpoint).json()

        # Analyze player picks
        red_card_flag, own_goal_flag, penalties_missed_flag = analyze_picks(element_ids_modified, live_gameweek_response)

        new_picks_data = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'picks': str(element_ids_modified),
            'red_card': red_card_flag,
            'own_goal': own_goal_flag,
            'missed_pen': penalties_missed_flag
        }

        # Append to picks_data using concat
        picks_data = pd.concat([picks_data, pd.DataFrame([new_picks_data])], ignore_index=True)

    # Update google sheets for new_gameweek_results and picks_data
    write_google_sheets_data(_gc, new_gameweek_results, gameweek_results_table, prod_google_sheet_key)
    write_google_sheets_data(_gc, picks_data, gameweek_teams_table, prod_google_sheet_key)
    
    return None

gameweek_results_update(_gc, current_gw, manager_details)

In [19]:
def fetch_google_sheets_data(gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    
def get_players_by_condition(df, condition_column):
    filtered_players = df[df[condition_column] == 'TRUE']
    return list(filtered_players['player_name'])


def auto_assign_drinks(_gc, gameweek_results_table, gameweek_teams_table, prod_google_sheet_key, current_gw):
    gameweek_results_table_df = fetch_google_sheets_data(_gc, gameweek_results_table, prod_google_sheet_key,  ["event", "points", "total_points", "event_transfers_cost", "points_on_bench"])
    gameweek_teams_table_df = fetch_google_sheets_data(_gc, gameweek_teams_table, prod_google_sheet_key,  ["event"])

    filtered_results = gameweek_results_table_df[gameweek_results_table_df["event"] == current_gw]
    filtered_teams = gameweek_teams_table_df[gameweek_teams_table_df["event"] == current_gw]

    lowest_points_player = filtered_results.sort_values(by=['points', 'total_points', 'player_name']).iloc[0]['player_name']
    created_date = (datetime.now() + timedelta(hours=2)).strftime("%d/%m/%y %H:%M:%S")
    deadline_date = (datetime.now() + timedelta(hours=2) + timedelta(days=7)).strftime("%d/%m/%y %H:00")

    # Get player names with red cards, own goals, and missed penalties
    red_card_players = get_players_by_condition(filtered_teams, 'red_card')
    own_goal_players = get_players_by_condition(filtered_teams, 'own_goal')
    missed_pen_players = get_players_by_condition(filtered_teams, 'missed_pen')

    red_card_drinks_data = {
        "event": [current_gw]*len(red_card_players),
        "nominator_name": ["auto"]*len(red_card_players),
        "drinker_name": red_card_players,
        "drink_type": ["red card"]*len(red_card_players),
        "nomination_created_date": [created_date]*len(red_card_players),
        "nomination_deadline_date": [deadline_date]*len(red_card_players),
        "nomination_completed_date": ["Not Completed"]*len(red_card_players),
    }

    own_goal_players_drinks_data = {
        "event": [current_gw]*len(own_goal_players),
        "nominator_name": ["auto"]*len(own_goal_players),
        "drinker_name": own_goal_players,
        "drink_type": ["own goal"]*len(own_goal_players),
        "nomination_created_date": [created_date]*len(own_goal_players),
        "nomination_deadline_date": [deadline_date]*len(own_goal_players),
        "nomination_completed_date": ["Not Completed"]*len(own_goal_players),
    }

    missed_pen_players_drinks_data = {
        "event": [current_gw]*len(missed_pen_players),
        "nominator_name": ["auto"]*len(missed_pen_players),
        "drinker_name": missed_pen_players,
        "drink_type": ["missed pen"]*len(missed_pen_players),
        "nomination_created_date": [created_date]*len(missed_pen_players),
        "nomination_deadline_date": [deadline_date]*len(missed_pen_players),
        "nomination_completed_date": ["Not Completed"]*len(missed_pen_players),
    }

    last_place_drinks_data = {
        "event": [current_gw],
        "nominator_name": ["auto"],
        "drinker_name": lowest_points_player,
        "drink_type": ["last place"],
        "nomination_created_date": [created_date],
        "nomination_deadline_date": [deadline_date],
        "nomination_completed_date": ["Not Completed"],
    }

    df1 = pd.DataFrame(red_card_drinks_data)
    df2 = pd.DataFrame(own_goal_players_drinks_data)
    df3 = pd.DataFrame(missed_pen_players_drinks_data)
    df4 = pd.DataFrame(last_place_drinks_data)

    concatenated_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

    write_google_sheets_data(_gc, concatenated_df, drinks_table, prod_google_sheet_key)

auto_assign_drinks(_gc, gameweek_results_table, gameweek_teams_table, prod_google_sheet_key, current_gw)